In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import pickle
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"



/Users/vyacheslav/Documents/projects/rubert-tiny/.venv39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/s3/hmpf_gw12rg9ltnxptr0q3pw0000gn/T/ipykernel_73981/1410809860.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
class YesNoModel(nn.Module):
    def set_config(self, config):
        pass

    def predict_group(self, group, txt):
        pass

    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=312, out_features=5)  # takes in 2 features (X), produces 5 features
        self.layer_2 = nn.Linear(in_features=5, out_features=1)  # takes in 5 features, produces 1 feature (y)

    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return self.layer_2(
            self.layer_1(
                x))  # computation goes through layer_1 first then the output of layer_1 goes through layer_2


In [13]:
with open('model_0.picle', 'rb') as f: 
    model_0 = pickle.load(f)
with open('model_rubertyni.picle', 'rb') as f: 
    modelrubertyni = pickle.load(f)
with open('tokenizer.picle', 'rb') as f: 
    tokenizer = pickle.load(f)
    
    

In [14]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

def make_predict(text, main_model, wv_model, wv_tokenizer):
    new_df = pd.DataFrame()
    new_df['DATA'] = [text]
    new_df['VEC'] = new_df['DATA'].apply(lambda  x : embed_bert_cls(x, wv_model, wv_tokenizer)) 
    X_train2 = torch.from_numpy(np.stack(new_df['VEC'].values)).type(torch.float)
    test_logits = main_model(X_train2).squeeze() 
    return int(torch.round(torch.sigmoid(test_logits)).tolist())
    

In [15]:
make_predict("ну да нет", model_0, modelrubertyni, tokenizer)

0

In [16]:
model_0.set_config({})